In [21]:
import numpy as np
from scipy import stats as st

from simulations import SimulationModel, heston_model_fun

RND_SEED = 42

In [22]:
def european_option_price(option_type: str, simulation_model: SimulationModel, K: float, S0: float,  r: float,
                          T: float, steps: int,
                          paths: int, confidence_level: float=0.95) -> float:
    """
    Parameters:
    option_type (str): 'call' or 'put'
    simulation_model (SimulationModel): a dataclass containing the model name, model function and extra arguments
    K (float): strike price
    S0 (float): initial stock price
    r (float): risk-free rate (%)
    T (float): time to maturity (years)
    steps (int): number of time steps
    paths (int): number of simulations
    confidence_level (float, optional): confidence level for confidence interval. Defaults to 0.95.
    Returns:
    mean (float): mean of option price
    std_error (float): standard deviation of option price
    ci_lower (float): lower limit of confidence interval
    ci_upper (float): upper limit of confidence interval
    """

    S = simulation_model.model(S0, r, T, steps, paths, **simulation_model.extra_args)
    S = S.T
    # Calculate the payoff for each path
    discount_factor = np.exp(-r * T)
    payoff = np.maximum(S[:, -1] - K, 0)
    std_error= np.std(payoff) * discount_factor / np.sqrt(steps)
    # Discount the payoff back to present value
    if option_type == 'call':
        payoff = np.maximum(S[:, -1] - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - S[:, -1], 0)
    option_price = discount_factor * np.mean(payoff)
    z_score = st.norm.ppf(1 - (1 - confidence_level) / 2)
    margin_error = z_score * std_error
    ci_lower = option_price - margin_error
    ci_upper = option_price + margin_error
    return option_price, std_error, ci_lower, ci_upper

In [23]:
sim_model = SimulationModel("Heston", heston_model_fun, dict(v0=0.01, kappa=1.0, theta=0.04, xi=0.1, rho=-0.7))

prices = european_option_price("call", sim_model, S0=98000, K=10800, r=3.0, T=100, steps=100,  paths=100000)
prices

(np.float64(99884.12787925405),
 np.float64(36318.86574902259),
 np.float64(28700.45905182445),
 np.float64(171067.79670668364))

In [ ]:
from crypto_exchange_utils import *
underlying_symbol = 'BTC'
exchange_name = 'binance'
df_underlying = load_underlying_ohlcv_from_csv(exchange_name, underlying_symbol, '1d', '2020-01-01 00:00:00Z', '2024-01-01 00:00:00Z')